In [ ]:
!pip install -q transformers[torch] datasets
!pip install -q bitsandbytes trl peft
!pip install flash-attn --no-build-isolation

In [ ]:
from datasets import load_dataset, DatasetDict


# Load the transformed dataset
dataset = load_dataset("json", data_files="all_instruct.jsonl")

# Convert to DatasetDict
raw_dataset = DatasetDict(
    {
        "train": dataset[
            "train"
        ]  # Assuming you want to use the loaded dataset as training data
    }
)

In [ ]:
from transformers import AutoTokenizer
from huggingface_hub import login

key = ""

login(key)


model_id = "mistralai/Mistral-7B-Instruct-v0.3"

tokenizer = AutoTokenizer.from_pretrained(model_id)

# set pad_token_id equal to the eos_token_id if not set
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

# Set reasonable default for models without max length
if tokenizer.model_max_length > 100_000:
    tokenizer.model_max_length = 2048

# Set chat template
DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"
tokenizer.chat_template = DEFAULT_CHAT_TEMPLATE

In [ ]:
import re
import random
from multiprocessing import cpu_count


def apply_chat_template(example, tokenizer):
    messages = example["messages"]
    # We add an empty system message if there is none
    if messages[0]["role"] != "system":
        messages.insert(0, {"role": "system", "content": ""})
    example["text"] = tokenizer.apply_chat_template(messages, tokenize=False)

    return example


column_names = list(raw_dataset["train"].features)
final_d = raw_dataset.map(
    apply_chat_template,
    num_proc=cpu_count(),
    fn_kwargs={"tokenizer": tokenizer},
    remove_columns=column_names,
    desc="Applying chat template",
)

# create the splits
train_dataset = final_d["train"]
# eval_dataset = raw_datasets["test"]

In [ ]:
import torch

device_map = {"": torch.cuda.current_device()} if torch.cuda.is_available() else None

model_kwargs = dict(
    attn_implementation="flash_attention_2",  # set this to True if your GPU supports it (Flash Attention drastically speeds up model computations)
    torch_dtype="auto",
    use_cache=False,  # set to False as we're going to use gradient checkpointing
    device_map=device_map,
)

In [ ]:
from trl import SFTTrainer
from peft import LoraConfig
from transformers import TrainingArguments


# path where the Trainer will save its checkpoints and logs
output_dir = "data/mistralinstruct-7b-sft-lora"

# based on config
training_args = TrainingArguments(
    bf16=True,  # specify bf16=True instead when training on GPUs that support bf16
    do_eval=True,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=128,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    learning_rate=2.0e-05,
    log_level="info",
    logging_steps=5,
    logging_strategy="steps",
    lr_scheduler_type="cosine",
    max_steps=-1,
    num_train_epochs=1,
    output_dir=output_dir,
    overwrite_output_dir=True,
    per_device_eval_batch_size=1,  # originally set to 8
    per_device_train_batch_size=1,  # originally set to 8
    push_to_hub=True,
    hub_model_id="mistralinstruct-7b-sft-lora",
    hub_strategy="every_save",
    report_to="tensorboard",
    save_strategy="no",
    save_total_limit=None,
    seed=42,
)


# based on config
peft_config = LoraConfig(
    r=64,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)

In [ ]:
trainer = SFTTrainer(
    model=model_id,
    model_init_kwargs=model_kwargs,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset,
    dataset_text_field="text",
    tokenizer=tokenizer,
    packing=True,
    peft_config=peft_config,
    max_seq_length=tokenizer.model_max_length,
)

In [ ]:
train_result = trainer.train()

In [ ]:
model_name = "mistralinstruct-7b-sft-lora"
trainer.push_to_hub(model_name)

In [ ]:
metrics = train_result.metrics
# max_train_samples = training_args.max_train_samples if training_args.max_train_samples is not None else len(train_dataset)
# metrics["train_samples"] = min(max_train_samples, len(train_dataset))
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(output_dir)
model = AutoModelForCausalLM.from_pretrained(
    output_dir, load_in_4bit=True, device_map="auto"
)

In [ ]:
import torch

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
]

# prepare the messages for the model
input_ids = tokenizer.apply_chat_template(
    messages, truncation=True, add_generation_prompt=True, return_tensors="pt"
).to("cuda")

# inference
outputs = model.generate(
    input_ids=input_ids,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])

In [ ]:
train_dataset.push_to_hub("belgian-law-qafrench-dataset")

In [ ]:
!zip -r /content/zipped.zip /content/data/mistralinstruct-7b-sft-lora